In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

import joblib

from ic.evaluation import evaluate_v2

# Avaliação

Aqui foram avaliados com os dados de teste cada um dos modelos gerados em [training.py](./training.py): `Gaussian Naive Bayes`, `Complement Naive Bayes` e `Hoeffding Tree` para cada arquivo (contendo tipos diferentes de ataque).

O código de avaliação também foi mudado para utilizar a matriz de confusão e métricas utilizando as funções do `scikit-learn`, o código pode ser visto em [ic/evaluation.py](../ic/evaluation.py) (ver `evaluate_v2).

In [2]:
model_names = ["naive_bayes", "complement_nb", "hoeffding"]

training_data = [
    ("../../data/Tuesday-WorkingHours.pcap_ISCX.processed.test.csv", "tue"),
    ("../../data/Wednesday-workingHours.pcap_ISCX.processed.test.csv", "wed"),
    ("../../data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.processed.test.csv", "thu_m"),
    ("../../data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.processed.test.csv", "thu_a"),
    ("../../data/Friday-WorkingHours-Morning.pcap_ISCX.processed.test.csv", "fri_m"),
    ("../../data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.processed.test.csv", "fri_a_ddos"),
    ("../../data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.processed.test.csv", "fri_a_portscan")
]

def model_path(model_name, kind):
    return "../../trained_models/"+model_name+"_v2_"+kind+".joblib"

No output da célula seguinte podemos ver as métricas de cada um dos modelos, treinados com cada arquivo (as "tags" que representam cada arquivo podem ser vistas no array `training_data` acima, para facilitar ver qual é qual no output)

In [3]:
for model_name in model_names:
    for test_csv_path, kind in training_data:
        print(model_name, kind)
        path = model_path(model_name, kind)
        model = joblib.load(path)
        results = evaluate_v2(
            model,
            test_csv_path
        )
        print(results, "\n---\n")

naive_bayes tue
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.9351332472535614, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[88357,     0],
       [ 6129,     0]]), 'lines_errored': 0} 
---

naive_bayes wed
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.49899391854674624, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[88780,     0],
       [89138,     0]]), 'lines_errored': 0} 
---

naive_bayes thu_m
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.9672333666686276, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[32884,     0],
       [ 1114,     0]]), 'lines_errored': 0} 
---

naive_bayes thu_a
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.999790224459828, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[61958,     0],
       [   13,     0]]), 'lines_errored': 0} 
---

naive_bayes fri_m
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.9734915216912574, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[35365,     0],
       [  963,     0]]), 'lines_errored'

---

## Ensembles

Também foram avaliadas ensembles usando os modelos acima (usando os 3 para cada arquivo) e duas estratégias diferentes: `any` (se qualquer um detectar um ataque, ela prevê como ataque) e `majority` (prevê de acordo com o voto da maioria).

O código por trás das ensembles pode ser visto em [ic/models/ensemble.py](../ic/models/ensemble.py)

No output da célula abaixo temos as métricas para cada tipo de ensemble (ver de acordo com "tag" do arquivo, conforme explicado nas células anteriores).

In [3]:
from ic.models.ensemble import any_pred, majority, Ensemble

ensemble_types = {
 'any': any_pred,
 'majority': majority
}

for test_csv_path, kind in training_data:
    for ensemble_type, ensemble_f in ensemble_types.items():
        print(ensemble_type, kind)
        model_paths = [ model_path(model_name, kind) for model_name in model_names ]
        ensemble = Ensemble(model_paths, ensemble_f)
        results = evaluate_v2(
            ensemble,
            test_csv_path
        )
        print(results, "\n---\n")
    

any tue
{'recall': 0.9990210474791973, 'precision': 0.1459525171624714, 'accuracy': 0.6207374637512436, 'false_alarm_rate': 0.4055026766413527, 'confusion_matrix': array([[52528, 35829],
       [    6,  6123]]), 'lines_errored': 0} 
---

majority tue
{'recall': 0.47071300375265135, 'precision': 1.0, 'accuracy': 0.965666871282518, 'false_alarm_rate': 0.0, 'confusion_matrix': array([[88357,     0],
       [ 3244,  2885]]), 'lines_errored': 0} 
---

any wed
{'recall': 0.9823868608225448, 'precision': 0.7855041263006818, 'accuracy': 0.856776717364179, 'false_alarm_rate': 0.26933994142824963, 'confusion_matrix': array([[64868, 23912],
       [ 1570, 87568]]), 'lines_errored': 0} 
---

majority wed
{'recall': 0.8709192488052233, 'precision': 0.9999742380915578, 'accuracy': 0.9353185175193066, 'false_alarm_rate': 2.2527596305474206e-05, 'confusion_matrix': array([[88778,     2],
       [11506, 77632]]), 'lines_errored': 0} 
---

any thu_m
{'recall': 0.8895870736086176, 'precision': 0.13579062